In [1]:
import pandas as pd
import numpy as np
import json
import warnings
import spacy
warnings.filterwarnings("ignore")
import random

import nltk
from nltk.stem import WordNetLemmatizer

import re

from spacy.tokens import DocBin

from tqdm import tqdm

In [2]:
train_data = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\train.csv")
features = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\features.csv")
patient_notes = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\patient_notes.csv")

In [3]:
new_train = train_data[(train_data.feature_num == 7)][['pn_num','annotation',"location","feature_num"]]
new_train

,pn_num,annotation,location,feature_num
7,16,[],[],7
20,41,['SOB'],['258 261'],7
33,46,['SOB'],['573 576'],7
46,82,['shortness of breath'],['284 303'],7
59,100,[],[],7
...,...,...,...,...
1242,2315,[],[],7
1255,2425,['SOB'],['330 333'],7
1268,2428,['short of breath'],['136 151'],7
1281,2430,['shortness of breath'],['284 303'],7


In [4]:
new_train = new_train[new_train.annotation != '[]']
new_train

,pn_num,annotation,location,feature_num
20,41,['SOB'],['258 261'],7
33,46,['SOB'],['573 576'],7
46,82,['shortness of breath'],['284 303'],7
72,161,['breathlessness'],['226 240'],7
98,211,['Short of breath'],['387 402'],7
...,...,...,...,...
1229,2308,"[""couldn't catch his breath""]",['386 411'],7
1255,2425,['SOB'],['330 333'],7
1268,2428,['short of breath'],['136 151'],7
1281,2430,['shortness of breath'],['284 303'],7


In [5]:
new_train["annotation"] = new_train.annotation.str.replace('[','')
new_train["annotation"] = new_train.annotation.str.replace(']','')
new_train["location"] = new_train.location.str.replace('[','')
new_train["location"] = new_train.location.str.replace(']','')

new_train

,pn_num,annotation,location,feature_num
20,41,'SOB','258 261',7
33,46,'SOB','573 576',7
46,82,'shortness of breath','284 303',7
72,161,'breathlessness','226 240',7
98,211,'Short of breath','387 402',7
...,...,...,...,...
1229,2308,"""couldn't catch his breath""",'386 411',7
1255,2425,'SOB','330 333',7
1268,2428,'short of breath','136 151',7
1281,2430,'shortness of breath','284 303',7


In [6]:
new_train = (new_train.set_index(['pn_num',"feature_num"]) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(new_train.columns, axis=1))

In [7]:
new_train

,pn_num,annotation,location,feature_num
0,41,'SOB','258 261',7
1,46,'SOB','573 576',7
2,82,'shortness of breath','284 303',7
3,161,'breathlessness','226 240',7
4,211,'Short of breath','387 402',7
...,...,...,...,...
70,2308,"""couldn't catch his breath""",'386 411',7
71,2425,'SOB','330 333',7
72,2428,'short of breath','136 151',7
73,2430,'shortness of breath','284 303',7


In [8]:
new_train = new_train.merge(patient_notes, on='pn_num', how='left')
new_train

,pn_num,annotation,location,feature_num,case_num,pn_history
0,41,'SOB','258 261',7,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...
1,46,'SOB','573 576',7,0,Mr. Cleveland is a 17yo M who was consented by...
2,82,'shortness of breath','284 303',7,0,17 yo M w/ no cardiac or arrhythmia PMH presen...
3,161,'breathlessness','226 240',7,0,"17 y/o M , Dillon Cleveland comes with c/o of ..."
4,211,'Short of breath','387 402',7,0,HPI: Patient is a 17 yo m with a c/o of palpit...
...,...,...,...,...,...,...
70,2308,"""couldn't catch his breath""",'386 411',7,0,17 yo male with no significant PMH presenting ...
71,2425,'SOB','330 333',7,0,17 yo CC palpitation \r\n-Started 3 months ago...
72,2428,'short of breath','136 151',7,0,dillon cleveland is a 17yo male with no PMH th...
73,2430,'shortness of breath','284 303',7,0,Mr. Cleveland is a 17 yo male who presents wit...


In [9]:
new_train["annotation"] = new_train["annotation"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))
new_train["annotation"] = new_train["annotation"].apply(lambda x: re.sub(r'''[']''','',x))
new_train["annotation"] = new_train["annotation"].apply(lambda x: x.strip())
new_train

,pn_num,annotation,location,feature_num,case_num,pn_history
0,41,SOB,'258 261',7,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...
1,46,SOB,'573 576',7,0,Mr. Cleveland is a 17yo M who was consented by...
2,82,shortness of breath,'284 303',7,0,17 yo M w/ no cardiac or arrhythmia PMH presen...
3,161,breathlessness,'226 240',7,0,"17 y/o M , Dillon Cleveland comes with c/o of ..."
4,211,Short of breath,'387 402',7,0,HPI: Patient is a 17 yo m with a c/o of palpit...
...,...,...,...,...,...,...
70,2308,couldnt catch his breath,'386 411',7,0,17 yo male with no significant PMH presenting ...
71,2425,SOB,'330 333',7,0,17 yo CC palpitation \r\n-Started 3 months ago...
72,2428,short of breath,'136 151',7,0,dillon cleveland is a 17yo male with no PMH th...
73,2430,shortness of breath,'284 303',7,0,Mr. Cleveland is a 17 yo male who presents wit...


In [10]:
new_train["New_pn_history"] = new_train["pn_history"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))
new_train["New_pn_history"] = new_train["New_pn_history"].apply(lambda x: re.sub(r'''[']''','',x))
new_train

,pn_num,annotation,location,feature_num,case_num,pn_history,New_pn_history
0,41,SOB,'258 261',7,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...
1,46,SOB,'573 576',7,0,Mr. Cleveland is a 17yo M who was consented by...,Mr. Cleveland is a 17yo M who was consented by...
2,82,shortness of breath,'284 303',7,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,17 yo M w no cardiac or arrhythmia PMH presen...
3,161,breathlessness,'226 240',7,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",17 y o M Dillon Cleveland comes with c o of ...
4,211,Short of breath,'387 402',7,0,HPI: Patient is a 17 yo m with a c/o of palpit...,HPI: Patient is a 17 yo m with a c o of palpit...
...,...,...,...,...,...,...,...
70,2308,couldnt catch his breath,'386 411',7,0,17 yo male with no significant PMH presenting ...,17 yo male with no significant PMH presenting ...
71,2425,SOB,'330 333',7,0,17 yo CC palpitation \r\n-Started 3 months ago...,17 yo CC palpitation -Started 3 months ago ...
72,2428,short of breath,'136 151',7,0,dillon cleveland is a 17yo male with no PMH th...,dillon cleveland is a 17yo male with no PMH th...
73,2430,shortness of breath,'284 303',7,0,Mr. Cleveland is a 17 yo male who presents wit...,Mr. Cleveland is a 17 yo male who presents wit...


In [11]:
new_train["start_location"] = new_train["location"].apply(lambda x: x.split()[0][1:])
new_train["end_location"] = new_train["location"].apply(lambda x: x.split()[-1][:-1])
new_train

,pn_num,annotation,location,feature_num,case_num,pn_history,New_pn_history,start_location,end_location
0,41,SOB,'258 261',7,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...,258,261
1,46,SOB,'573 576',7,0,Mr. Cleveland is a 17yo M who was consented by...,Mr. Cleveland is a 17yo M who was consented by...,573,576
2,82,shortness of breath,'284 303',7,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,17 yo M w no cardiac or arrhythmia PMH presen...,284,303
3,161,breathlessness,'226 240',7,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",17 y o M Dillon Cleveland comes with c o of ...,226,240
4,211,Short of breath,'387 402',7,0,HPI: Patient is a 17 yo m with a c/o of palpit...,HPI: Patient is a 17 yo m with a c o of palpit...,387,402
...,...,...,...,...,...,...,...,...,...
70,2308,couldnt catch his breath,'386 411',7,0,17 yo male with no significant PMH presenting ...,17 yo male with no significant PMH presenting ...,386,411
71,2425,SOB,'330 333',7,0,17 yo CC palpitation \r\n-Started 3 months ago...,17 yo CC palpitation -Started 3 months ago ...,330,333
72,2428,short of breath,'136 151',7,0,dillon cleveland is a 17yo male with no PMH th...,dillon cleveland is a 17yo male with no PMH th...,136,151
73,2430,shortness of breath,'284 303',7,0,Mr. Cleveland is a 17 yo male who presents wit...,Mr. Cleveland is a 17 yo male who presents wit...,284,303


In [12]:
new_train["New_annotation"] = new_train.apply(lambda x : x.annotation.replace("SOB", "short of breath"), axis = 1)
new_train

,pn_num,annotation,location,feature_num,case_num,pn_history,New_pn_history,start_location,end_location,New_annotation
0,41,SOB,'258 261',7,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...,258,261,short of breath
1,46,SOB,'573 576',7,0,Mr. Cleveland is a 17yo M who was consented by...,Mr. Cleveland is a 17yo M who was consented by...,573,576,short of breath
2,82,shortness of breath,'284 303',7,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,17 yo M w no cardiac or arrhythmia PMH presen...,284,303,shortness of breath
3,161,breathlessness,'226 240',7,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",17 y o M Dillon Cleveland comes with c o of ...,226,240,breathlessness
4,211,Short of breath,'387 402',7,0,HPI: Patient is a 17 yo m with a c/o of palpit...,HPI: Patient is a 17 yo m with a c o of palpit...,387,402,Short of breath
...,...,...,...,...,...,...,...,...,...,...
70,2308,couldnt catch his breath,'386 411',7,0,17 yo male with no significant PMH presenting ...,17 yo male with no significant PMH presenting ...,386,411,couldnt catch his breath
71,2425,SOB,'330 333',7,0,17 yo CC palpitation \r\n-Started 3 months ago...,17 yo CC palpitation -Started 3 months ago ...,330,333,short of breath
72,2428,short of breath,'136 151',7,0,dillon cleveland is a 17yo male with no PMH th...,dillon cleveland is a 17yo male with no PMH th...,136,151,short of breath
73,2430,shortness of breath,'284 303',7,0,Mr. Cleveland is a 17 yo male who presents wit...,Mr. Cleveland is a 17 yo male who presents wit...,284,303,shortness of breath


In [13]:
new_train["New_pn_history"] = new_train.apply(lambda x : x.New_pn_history.replace("SOB", "short of breath"), axis = 1)
new_train

,pn_num,annotation,location,feature_num,case_num,pn_history,New_pn_history,start_location,end_location,New_annotation
0,41,SOB,'258 261',7,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...,258,261,short of breath
1,46,SOB,'573 576',7,0,Mr. Cleveland is a 17yo M who was consented by...,Mr. Cleveland is a 17yo M who was consented by...,573,576,short of breath
2,82,shortness of breath,'284 303',7,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,17 yo M w no cardiac or arrhythmia PMH presen...,284,303,shortness of breath
3,161,breathlessness,'226 240',7,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",17 y o M Dillon Cleveland comes with c o of ...,226,240,breathlessness
4,211,Short of breath,'387 402',7,0,HPI: Patient is a 17 yo m with a c/o of palpit...,HPI: Patient is a 17 yo m with a c o of palpit...,387,402,Short of breath
...,...,...,...,...,...,...,...,...,...,...
70,2308,couldnt catch his breath,'386 411',7,0,17 yo male with no significant PMH presenting ...,17 yo male with no significant PMH presenting ...,386,411,couldnt catch his breath
71,2425,SOB,'330 333',7,0,17 yo CC palpitation \r\n-Started 3 months ago...,17 yo CC palpitation -Started 3 months ago ...,330,333,short of breath
72,2428,short of breath,'136 151',7,0,dillon cleveland is a 17yo male with no PMH th...,dillon cleveland is a 17yo male with no PMH th...,136,151,short of breath
73,2430,shortness of breath,'284 303',7,0,Mr. Cleveland is a 17 yo male who presents wit...,Mr. Cleveland is a 17 yo male who presents wit...,284,303,shortness of breath


In [14]:
def word_lemm(sentence):
    word_list = nltk.word_tokenize(sentence)
    
    lemmaztier = WordNetLemmatizer()
    lemmatized_output = ' '.join([lemmaztier.lemmatize(w) for w in word_list])
    return(lemmatized_output)

In [15]:
new_train["New_annotation"] = new_train["New_annotation"].apply(word_lemm)
new_train["New_pn_history"] = new_train["New_pn_history"].apply(word_lemm)
new_train

,pn_num,annotation,location,feature_num,case_num,pn_history,New_pn_history,start_location,end_location,New_annotation
0,41,SOB,'258 261',7,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...,258,261,short of breath
1,46,SOB,'573 576',7,0,Mr. Cleveland is a 17yo M who was consented by...,Mr. Cleveland is a 17yo M who wa consented by ...,573,576,short of breath
2,82,shortness of breath,'284 303',7,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,17 yo M w no cardiac or arrhythmia PMH present...,284,303,shortness of breath
3,161,breathlessness,'226 240',7,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",17 y o M Dillon Cleveland come with c o of pal...,226,240,breathlessness
4,211,Short of breath,'387 402',7,0,HPI: Patient is a 17 yo m with a c/o of palpit...,HPI : Patient is a 17 yo m with a c o of palpi...,387,402,Short of breath
...,...,...,...,...,...,...,...,...,...,...
70,2308,couldnt catch his breath,'386 411',7,0,17 yo male with no significant PMH presenting ...,17 yo male with no significant PMH presenting ...,386,411,couldnt catch his breath
71,2425,SOB,'330 333',7,0,17 yo CC palpitation \r\n-Started 3 months ago...,17 yo CC palpitation -Started 3 month ago ha e...,330,333,short of breath
72,2428,short of breath,'136 151',7,0,dillon cleveland is a 17yo male with no PMH th...,dillon cleveland is a 17yo male with no PMH th...,136,151,short of breath
73,2430,shortness of breath,'284 303',7,0,Mr. Cleveland is a 17 yo male who presents wit...,Mr. Cleveland is a 17 yo male who present with...,284,303,shortness of breath


In [16]:
new_train["start_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history).start(),axis =1)
new_train["end_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history).end(),axis =1)
new_train

,pn_num,annotation,location,feature_num,case_num,pn_history,New_pn_history,start_location,end_location,New_annotation
0,41,SOB,'258 261',7,0,17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,17 Y O M CAME TO THE CLINIC C O HEART POUNDING...,261,276,short of breath
1,46,SOB,'573 576',7,0,Mr. Cleveland is a 17yo M who was consented by...,Mr. Cleveland is a 17yo M who wa consented by ...,559,574,short of breath
2,82,shortness of breath,'284 303',7,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,17 yo M w no cardiac or arrhythmia PMH present...,278,297,shortness of breath
3,161,breathlessness,'226 240',7,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",17 y o M Dillon Cleveland come with c o of pal...,220,234,breathlessness
4,211,Short of breath,'387 402',7,0,HPI: Patient is a 17 yo m with a c/o of palpit...,HPI : Patient is a 17 yo m with a c o of palpi...,382,397,Short of breath
...,...,...,...,...,...,...,...,...,...,...
70,2308,couldnt catch his breath,'386 411',7,0,17 yo male with no significant PMH presenting ...,17 yo male with no significant PMH presenting ...,378,402,couldnt catch his breath
71,2425,SOB,'330 333',7,0,17 yo CC palpitation \r\n-Started 3 months ago...,17 yo CC palpitation -Started 3 month ago ha e...,309,324,short of breath
72,2428,short of breath,'136 151',7,0,dillon cleveland is a 17yo male with no PMH th...,dillon cleveland is a 17yo male with no PMH th...,132,147,short of breath
73,2430,shortness of breath,'284 303',7,0,Mr. Cleveland is a 17 yo male who presents wit...,Mr. Cleveland is a 17 yo male who present with...,277,296,shortness of breath


In [17]:
sep_combined_case0  = new_train.copy()

In [18]:
pn_Num_list = list(sep_combined_case0.pn_num.unique())

In [19]:
final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in sep_combined_case0[sep_combined_case0.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = sep_combined_case0[sep_combined_case0.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list

In [20]:
final_train[26]

('HPI- 17 YO M WITH C O HEART PALPITATIONS FOR 3 MONTHS WHICH WAS SUDDEN IN ONSET INTERMITTENT AND PROGRESSIVE . HE HAD A SEVERE EPISODE 2 DAYS AGO WHERE HE HAD CHEST TIGHTNESS AND short of breath . HE IS STRESSED OUT ABOUT COLLEGE AND IS TAKING ADY WHICH HIS FRIEND PROVOIDED TO HIM . HE ALSO CONSUMES 3 TO 4 CUPS OF COFFEE EACH DAY . DENIES FEVER CHEST PAIN SLEEP CHANGES COUGH SKIN CHANGES CHANGES IN WEIGHT OR APPETITE TREMORS short of breath ON EXERTION CHANGES IN URINARY OR BOWEL HABITS . PMH PSH- NONE ALLERGIES- NKA MEDS- NONE FH- MOTHER HAS THYROID PROBLEMS AND FATHER HAD MI SH- DRINKS 3 TO 4 BEERS WEEKEND DENIES TOBACCO SMOKED POT ONCE PLAYS BASKETBALL AND DOES EXERCISE . SEXUAL- MONOGAMOUS AND USES CONDOMS .',
 {'entities': [(179, 194, 'Feature_7')]})

In [21]:
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [22]:
# Opening JSON file
f = open("sample.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [23]:
nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [24]:
train = data[:int(len(data)*0.8)]
validation = data[int(len(data)*0.8):]

In [25]:
len(train),len(validation)

(58, 15)

In [26]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 58/58 [00:00<00:00, 772.59it/s]


In [27]:
#validation data set
camp_train = create_training(validation)
camp_train.to_disk("nbme_validation.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 882.52it/s]


In [28]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-04-06 00:04:20.990424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-06 00:04:20.990461: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [29]:
!python -m spacy train config.cfg --output ./output

2022-04-06 00:04:24.593058: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-06 00:04:24.593099: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-04-06 00:04:26,922] [INFO] Set up nlp object from config
[2022-04-06 00:04:26,928] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-04-06 00:04:26,931] [INFO] Created vocabulary
[2022-04-06 00:04:26,931] [INFO] Finished initializing nlp object
[2022-04-06 00:04:27,277] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     60.83    0.00    0.00    0.00    0.00
  3     200       2525.55   2040.32   87.50   82.35   93.33    0.87
  6     400         11.05     17.37   87.50   82.35   93.33    0.87
 10     600          2.04      3.11   90.32   87.50   93.33    0.90
 13     800          2.42      3.52   90.32   87.50   93.33    0.90
 17    1000          1.78      2.31   90.32   87.50   93.33    0.90
 20    1200          0.00      0.00   90.32   87.50   93.33    0.90
 24    1400          0.00      0.00   90.32   87.50   93.33    0.90
 27    1600          